In [2]:
import ast
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
data = pd.read_csv("./data.csv")
nn_params = pd.read_csv("./NN.csv")

In [4]:
data

,Age,Income,Loan_Amount,Credit_History,Employment_Years,Default_Payment
0,35,45000,15000,1,5,1
1,40,30000,20000,0,10,0
2,50,55000,10000,1,15,0
3,28,32000,12000,0,2,1
4,45,40000,18000,1,8,0
5,30,28000,9000,1,3,1
6,38,50000,17000,1,12,0
7,33,48000,16000,0,6,1
8,47,39000,14000,1,9,0
9,29,36000,11000,0,4,1


In [5]:
nn_params

,Layer,Neuron,Weights,Bias
0,Input to Hidden 1,Neuron 1,"[0.1, 0.2, -0.3, 0.4, 0.5]",0.10
1,Input to Hidden 1,Neuron 2,"[0.11, 0.21, -0.29, 0.41, 0.49]",0.12
2,Input to Hidden 1,Neuron 3,"[0.12, 0.22, -0.27, 0.42, 0.48]",0.14
3,Input to Hidden 1,Neuron 4,"[0.13, 0.23, -0.27, 0.43, 0.47]",0.16
4,Input to Hidden 1,Neuron 5,"[0.14, 0.24, -0.26, 0.44, 0.46]",0.18
5,Input to Hidden 1,Neuron 6,"[0.15, 0.25, -0.25, 0.45, 0.45]",0.20
6,Input to Hidden 1,Neuron 7,"[0.16, 0.26, -0.24, 0.46, 0.44]",0.22
7,Input to Hidden 1,Neuron 8,"[0.17, 0.27, -0.22, 0.47, 0.43]",0.24
8,Input to Hidden 1,Neuron 9,"[0.18, 0.28, -0.21, 0.48, 0.42]",0.26
9,Input to Hidden 1,Neuron 10,"[0.19, 0.29, -0.21, 0.49, 0.41]",0.28


1) მონაცემების გამოყენებმადე მოახდინეთ მონაცემების სტანდარტიზაცია(გულისხმობს მონაცემების წარმოდგენას 0-დან 1 შუალედში), ფორმულა: 

In [6]:
X = data[["Age", "Income", "Loan_Amount", "Credit_History", "Employment_Years"]].to_numpy()
X = (X - X.min()) / (X.max() - X.min())
y = data["Default_Payment"].to_numpy()

In [7]:
def relu(x):
    return x * (x > 0)

def d_relu(x):
    return 1. * (x >= 0)

def sigmoid(z):
  return 1 / (1 + np.exp(-z))

def d_sigmoid(z):
  return sigmoid(z) * (1 - sigmoid(z))

In [8]:
def get_parameters(nn_params: pd.DataFrame):
    L1_params = nn_params[nn_params["Layer"] == "Input to Hidden 1"]
    L2_params = nn_params[nn_params["Layer"] == "Hidden 1 to Hidden 2"]
    L3_params = nn_params[nn_params["Layer"] == "Hidden 2 to Output"]
    w1 = np.array(list(map(ast.literal_eval, L1_params["Weights"].to_numpy())))
    b1 = L1_params["Bias"].to_numpy().reshape(10, 1)

    w2 = np.array(list(map(ast.literal_eval, L2_params["Weights"].to_numpy())))
    b2 = L2_params["Bias"].to_numpy().reshape(5, 1)

    w3 = np.array(list(map(ast.literal_eval, L3_params["Weights"].to_numpy())))
    b3 = L3_params["Bias"].to_numpy()

    return w1, w2, w3, b1, b2, b3

In [9]:
def predict(xi, w1, w2, w3, b1, b2, b3):
    # input -> hidden 1
    z1 = np.dot(w1, xi) + b1
    a1 = relu(z1)
    # hidden 1 -> hidden 2
    z2 = np.dot(w2, a1) + b2
    a2 = relu(z2) 
    # hidden 2 -> output
    z3 = np.dot(w3, a2) + b3
    a3 = sigmoid(z3)
    return a3

In [10]:
def cross_entropy(a3, yi):
    return -1 * (y * np.log(a3) + (1 - yi) * np.log(1-a3))

def d_cross_entropy(a3, yi):
    return  (a3 - yi) / (a3*(1-a3))

In [11]:
def nn(X, y, epochs, learn_rate):
    w1, w2, w3, b1, b2, b3 = get_parameters(nn_params)
    for epoch in range(epochs):
        for xi, yi in zip(X, y):
            xi.shape += (1,)
            yi = yi.reshape(1, 1)

            # input -> hidden 1
            z1 = np.dot(w1, xi) + b1
            a1 = relu(z1)
            # hidden 1 -> hidden 2
            z2 = np.dot(w2, a1) + b2
            a2 = relu(z2) 
            # hidden 2 -> output
            z3 = np.dot(w3, a2) + b3
            a3 = sigmoid(z3)

            # compute cost
            c = cross_entropy(a3, yi)

            # backpropagation
            # output -> hidden 2
            dc_da3 = d_cross_entropy(a3, yi)
            da3_dz3 = d_sigmoid(z3)
            dz3_dw3 = a2 
            # hidden 2 -> hidden 1
            da2_dz2 = d_relu(z2)
            dz2_dw2 = a1
            # hidden 1 -> input        
            da1_dz1 = d_relu(z1)
            dz1_dw1 = xi

            # update weight/bias with grad
            w3 = w3 - (learn_rate * dc_da3 * da3_dz3 * dz3_dw3).T
            b3 = b3 - learn_rate * np.sum(dc_da3 * da3_dz3, axis=0)
            w2 = w2 - learn_rate * da2_dz2 * dz2_dw2.T
            b2 = b2 - learn_rate * da2_dz2
            w1 = w1 - learn_rate * da1_dz1 * dz1_dw1.T
            b1 = b1 - learn_rate * da1_dz1

        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"Epoch : {epoch+1} - Cost: {round(c[0][0], 2)}%")
    return w1, w2, w3, b1, b2, b3, c


In [12]:
LEARNING_RATE = 0.01
test = np.array([29, 42000, 12000, 1, 5]).reshape(5, 1)

2) ზემოთ მოცემული მონაცემების საფუძველზე პითონის გარემოში ააგეთ ნეირონული ქსელი და დაატრენინგეთ მხოლოდ პირველი ორ მონაცემზე ერთი სრული ეპოქისთვის. იპოვეთ ახალი მომხარებლის საპროგნოზო მნიშვნელობა.

In [13]:
w1, w2, w3, b1, b2, b3, c = nn(X=X[:2], y=y[:2], epochs=1, learn_rate=LEARNING_RATE)
predict(test, w1, w2, w3, b1, b2, b3)

Epoch : 1 - Cost: 1.41%


array([[1.]])

3) ქსელი დაატრენინგეთ სრულ 10-ვე მონაცემზე ორი ეპოქის შემთხვევაში და იპოვეთ ახალი მომხარებლის საპროგნოზო მნიშვნელობა.

In [14]:
w1, w2, w3, b1, b2, b3, c = nn(X=X, y=y, epochs=2, learn_rate=LEARNING_RATE)
predict(test, w1, w2, w3, b1, b2, b3)

Epoch : 1 - Cost: 0.62%


array([[1.]])

4) ქსელი დაატრენინგეთ სრულ 10 მონაცემზე 100 ეპოქის შემთხვეაში და იპივეთ ახალი
მომხმარებლის საპროგნოზო მნიშვნელობა.

In [15]:
w1, w2, w3, b1, b2, b3, c = nn(X=X, y=y, epochs=100, learn_rate=LEARNING_RATE)
predict(test, w1, w2, w3, b1, b2, b3)

Epoch : 1 - Cost: 0.62%
Epoch : 10 - Cost: 0.68%
Epoch : 20 - Cost: 0.68%
Epoch : 30 - Cost: 0.69%
Epoch : 40 - Cost: 0.69%
Epoch : 50 - Cost: 0.69%
Epoch : 60 - Cost: 0.69%
Epoch : 70 - Cost: 0.69%
Epoch : 80 - Cost: 0.69%
Epoch : 90 - Cost: 0.69%
Epoch : 100 - Cost: 0.69%


array([[0.50122847]])

5) ნეირონული ქსელის ასაგებად გამოიყენეთ რომელიმე ბიბლიოთეკა და შეადარეთ თქვენს
მიერ დაწერილ ალგორითმს, ბიბლიოთეკის მიერ მიღებული შედეგი.

In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(X[0].shape),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(5, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

2024-12-31 15:26:50.515511: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
model.fit(X, y, epochs=100)

Epoch 1/100


/home/kuph/deep_learning/venv/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:750: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step - accuracy: 0.5000 - loss: 0.6998
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5000 - loss: 0.6989
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.6980
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5000 - loss: 0.6972
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5000 - loss: 0.6964
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 0.6956
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5000 - loss: 0.6948
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5000 - loss: 0.6941
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5000 - loss: 0.6934
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5000 - loss: 0.6927
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 0.6920
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 0.6914


In [19]:
keras_loss = model.evaluate(X, y)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.6000 - loss: 0.6755


In [20]:
print("Custom NN Loss: ", c[0,0])
print("Keras Loss: ", keras_loss)

Custom NN Loss:  0.6931903372846259
Keras Loss:  0.6754850149154663
